In [4]:
# pip install pandas nltk pyodbc sqlalchemy

import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
# Download the VADER lexicon for sentiment analysis if not already present.

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


True

In [18]:
# Define a function to fetch data from a SQL database using a SQL query

def fetch_data_from_sql():
    conn_str=(
    "Driver=Sql Server;"
    "Server=LAPTOP-RM1GFV65\MSSQL;"
    "Database=PortfolioProject_MarketingAnalytics;"
    "Trusted_Connection=yes;"
    )
    
    conn = pyodbc.connect(conn_str)
    
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"
    
    df = pd.read_sql(query,conn)
    
    conn.close()
    
    return df


In [19]:
# Fetch the customer reviews data from the SQL database

customer_reviews_df = fetch_data_from_sql()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [20]:
# Initialize the VADER sentiment intensity analyzer for analyzing the sentiment of text data


sia = SentimentIntensityAnalyzer()

In [21]:
# Define a function to calculate sentiment scores using VADER


def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [22]:
# Define a function to categorize sentiment using both the sentiment score and the review rating


def categorize_sentiment(score, rating):

    if score > 0.05: 
        if rating >= 4:
            return 'Positive' 
        elif rating == 3:
            return 'Mixed Positive' 
        else:
            return 'Mixed Negative' 
    elif score < -0.05:  
        if rating <= 2:
            return 'Negative'  
        elif rating == 3:
            return 'Mixed Negative'  
        else:
            return 'Mixed Positive' 
    else:  
        if rating >= 4:
            return 'Positive'  
        elif rating <= 2:
            return 'Negative'  
        else:
            return 'Neutral' 

In [23]:
# Define a function to bucket sentiment scores into text ranges


def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0' 
    else:
        return '-1.0 to -0.5' 

In [24]:
# Apply sentiment analysis to calculate sentiment scores for each review


customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

In [25]:
# Apply sentiment categorization using both text and rating


customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row : categorize_sentiment(row['SentimentScore'],row['Rating']),axis=1)

In [26]:
# Apply sentiment bucketing to categorize scores into defined ranges


customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

In [27]:
# Display the first few rows of the DataFrame with sentiment scores, categories, and buckets


print(customer_reviews_df.head())

   ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0         1          77         18  2023-12-23       3   
1         2          80         19  2024-12-25       5   
2         3          50         13  2025-01-26       4   
3         4          78         15  2025-04-21       3   
4         5          64          2  2023-07-16       3   

                                 ReviewText  SentimentScore SentimentCategory  \
0   Average  experience,  nothing  special.         -0.3089    Mixed Negative   
1            The  quality  is    top-notch.          0.0000          Positive   
2   Five  stars  for  the  quick  delivery.          0.0000          Positive   
3  Good  quality,  but  could  be  cheaper.          0.2382    Mixed Positive   
4   Average  experience,  nothing  special.         -0.3089    Mixed Negative   

  SentimentBucket  
0    -0.49 to 0.0  
1     0.0 to 0.49  
2     0.0 to 0.49  
3     0.0 to 0.49  
4    -0.49 to 0.0  


In [28]:
# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file


customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)